## Classifier

In [ ]:
import tensorflow as tf

import pandas as pd
from keras.layers import Input, Dense,LSTM
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import numpy as np
import math


from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
!rm * -rf

In [ ]:
!wget https://beaterblank.github.io/RESUME/Names_dataset.csv

--2022-07-26 12:31:40--  https://beaterblank.github.io/RESUME/Names_dataset.csv
Resolving beaterblank.github.io (beaterblank.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to beaterblank.github.io (beaterblank.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1394933 (1.3M) [text/csv]
Saving to: ‘Names_dataset.csv’

Names_dataset.csv   100%[===================>]   1.33M  --.-KB/s    in 0.05s   

2022-07-26 12:31:40 (24.2 MB/s) - ‘Names_dataset.csv’ saved [1394933/1394933]



In [ ]:
name = pd.read_csv("Names_dataset.csv")

In [ ]:
name.columns

Index(['name', 'gender'], dtype='object')

In [ ]:
fe = name[name.gender=='f'][:len(name[name.gender=='m'])].values
ma = name[name.gender=='m'][:len(name[name.gender=='m'])].values

In [ ]:
name=np.vstack((fe,ma))
name.shape

(98520, 2)

In [ ]:
name[900,0]

'cheriah'

In [ ]:
a=[]
for i in range(len(name)):
    try:
        if(math.isnan(name[i,0])):
            a.append(i)
    except:
        pass
name=np.delete(name,np.array(a),axis=0)

import re
for i in range(len(name)):
    name[i][0]=name[i][0].upper()
    name[i][0]=re.sub('[^ A-Z]+',' ',name[i][0])

a=[]
for i in range(len(name)):
    try:
        if(len(name[i,0])>=20):
            a.append(i)
    except:
        pass
name=np.delete(name,np.array(a),axis=0)

In [ ]:
name.shape

(97952, 2)

In [ ]:
name

array([['ALFIYA', 'f'],
       ['HENRYKA', 'f'],
       ['PREETI', 'f'],
       ...,
       ['DALSHAWN', 'm'],
       ['HILLARY', 'm'],
       ['ASHISH', 'm']], dtype=object)

In [ ]:
# Padding to make constant length
a=[]
for i in range(len(name)):
    try:
        if(len(name[i][0])<20):
            name[i][0]=(20-len(name[i][0]))//2*'0'+name[i][0]+(20-len(name[i][0]))//2*'0'+len(name[i][0])%2*'0'
    except:
        pass

In [ ]:
name.shape

(97952, 2)

In [ ]:
name

array([['0000000ALFIYA0000000', 'f'],
       ['000000HENRYKA0000000', 'f'],
       ['0000000PREETI0000000', 'f'],
       ...,
       ['000000DALSHAWN000000', 'm'],
       ['000000HILLARY0000000', 'm'],
       ['0000000ASHISH0000000', 'm']], dtype=object)

In [ ]:
x=name[:,0]
y=name[:,1]
y

array(['f', 'f', 'f', ..., 'm', 'm', 'm'], dtype=object)

In [ ]:
jn="".join(x)

lbl=LabelEncoder()
one=OneHotEncoder()

y=lbl.fit_transform(y)
y=one.fit_transform(y.reshape(len(y),1)).toarray()

In [ ]:
x

array(['0000000ALFIYA0000000', '000000HENRYKA0000000',
       '0000000PREETI0000000', ..., '000000DALSHAWN000000',
       '000000HILLARY0000000', '0000000ASHISH0000000'], dtype=object)

In [ ]:
y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

chars will have all the set of charecters in the whole names db in our case it should ideally be A-Z so we can change that to that and also we are encoding it using the array location mapping is the encoder

In [ ]:
chars = sorted(list(set(jn)))
mapping = dict((c, i) for i, c in enumerate(chars))

names = []
for line in x:
    encoded_seq = [mapping[char] for char in line]
    names.append(encoded_seq)

In [ ]:
len(names[0])

20

In [ ]:
x=np.array(names).reshape(len(names),20,1)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3, random_state=42)

In [ ]:
#Model
inp=Input(shape=(20,1))
lstm=LSTM(30,return_sequences=True)(inp)
lstm=LSTM(70,return_sequences=True)(lstm)
lstm=LSTM(30)(lstm)
out=Dense(2,activation='softmax')(lstm)

In [ ]:
model=Model(inp,out)
print(model.summary())
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 1)]           0         
                                                                 
 lstm (LSTM)                 (None, 20, 30)            3840      
                                                                 
 lstm_1 (LSTM)               (None, 20, 70)            28280     
                                                                 
 lstm_2 (LSTM)               (None, 30)                12120     
                                                                 
 dense (Dense)               (None, 2)                 62        
                                                                 
Total params: 44,302
Trainable params: 44,302
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(x_train, y_train, batch_size=50, epochs=50, verbose=1, validation_data=(x_test,y_test)) 

In [ ]:
model.save_weights('./weights')